# Evaluation using BERTscore

[Source](https://huggingface.co/spaces/evaluate-metric/bertscore)

In [1]:
import pandas as pd

df = pd.read_csv("../agent/predictions.csv")
df = df.drop(columns=['debias_reasoning'])
df = df.rename(columns={"biases": "biases_detected"})
df.head()

,input,biases_detected,scores,output,index
0,Estimada comunidad beauchefiana: ¿Tienes papel...,UNBIASED,1.0,UNBIASED,0
1,Desde hoy y hasta el 19 de diciembre puedes de...,GENERIC_PRONOUNS,0.8,Desde hoy y hasta el 19 de diciembre puedes de...,1
2,Revisa en el afiche qué tipo de papeles puedes...,UNBIASED,1.0,UNBIASED,2
3,¡Les esperamos!,UNBIASED,1.0,UNBIASED,3
4,Estimada Comunidad: La Subdirección de Puebl...,EXCLUSIONARY_TERMS,0.8,Estimada Comunidad: La Subdirección de Puebl...,4


In [2]:
df['biases_detected'] = df['biases_detected'].fillna('UNBIASED')
df['output'] = df['output'].fillna('UNBIASED')
len(df)

819

In [3]:
df_causal = pd.read_csv("../../data/processed/20231220_metrics_CAUSAL.csv")
df = pd.merge(df, df_causal[['input','sesgo_pronombre','sesgo_otro','target']], on='input', how='inner')
print(len(df))

748


In [4]:
df.head()

,input,biases_detected,scores,output,index,sesgo_pronombre,sesgo_otro,target
0,Estimada comunidad beauchefiana: ¿Tienes papel...,UNBIASED,1.0,UNBIASED,0,NO,NO,['Estimada comunidad beauchefiana: ¿Tienes pap...
1,Desde hoy y hasta el 19 de diciembre puedes de...,GENERIC_PRONOUNS,0.8,Desde hoy y hasta el 19 de diciembre puedes de...,1,NaN,NaN,['Desde hoy y hasta el 19 de diciembre puedes ...
2,Revisa en el afiche qué tipo de papeles puedes...,UNBIASED,1.0,UNBIASED,2,NaN,NaN,['Revisa en el afiche qué tipo de papeles pued...
3,Estimada Comunidad: La Subdirección de Puebl...,EXCLUSIONARY_TERMS,0.8,Estimada Comunidad: La Subdirección de Puebl...,4,NO,NO,['Estimada Comunidad: La Subdirección de Pue...
4,"Postulaciones, labores y más información en: ...",UNBIASED,1.0,UNBIASED,5,NaN,NaN,"['Postulaciones, labores y más información en:..."


### Example

There are potentially many options for replacing a biased text. We will consider the highest similarity among all candidate options.

In [5]:
references = ['Se informa a la comunidad estudiantil que quienes NO asistan al proceso de captura fotográfica correspondiente a la retarjetización de la Tarjeta Nacional Estudiantil, quedarán sin esta nueva versión desde el próximo año.']
predictions = ['Se informa a toda la comunidad estudiantil que las personas que NO asistan al proceso de captura fotográfica correspondiente a la retarjetización de la Tarjeta Nacional Estudiantil, no podrán obtener esta nueva versión desde el próximo año.' for _ in references]

In [6]:
from evaluate import load
bertscore = load("bertscore")

ModuleNotFoundError: No module named 'evaluate'

In [16]:
results = bertscore.compute(predictions=predictions, references=references, model_type="bert-base-multilingual-cased")
print(results)

{'precision': [0.9533587098121643], 'recall': [0.9690830707550049], 'f1': [0.9611566066741943], 'hashcode': 'bert-base-multilingual-cased_L9_no-idf_version=0.3.12(hug_trans=4.47.1)'}


Full evaluation

In [17]:
merged_df['version_sin_sesgo']

0                                                    NaN
1                                                    NaN
2      Unidad de Calidad de Vida Estudiantil les invi...
3      Es necesario el compromiso de acompañar a una/...
4                                                    NaN
                             ...                        
943                                                  NaN
944                                                  NaN
945                                                  NaN
946    El Centro de Estudiantes de Ingeniería (CEI) l...
947                                                  NaN
Name: version_sin_sesgo, Length: 948, dtype: object

In [18]:
merged_df = merged_df.dropna(subset=['version_sin_sesgo'])
print(len(merged_df))

333


In [21]:
results = bertscore.compute(predictions=list(merged_df['output']), references=list(merged_df['version_sin_sesgo']), model_type="bert-base-multilingual-cased")

In [23]:
merged_df['precision'] = results['precision']
merged_df['recall'] = results['recall']
merged_df['f1'] = results['f1']

In [24]:
len(merged_df)

333

In [25]:
def filter_best_scores(df, id_column, score_column):
    """
    Reduces the dataframe to only the rows with the best score for each ID.

    Parameters:
    - df (pd.DataFrame): The original dataframe
    - id_column (str): Name of the column containing unique IDs
    - score_column (str): Name of the column containing the scores

    Returns:
    - pd.DataFrame: A filtered dataframe with the best score per ID
    """
    # Find the maximum score for each ID_row
    best_scores = df.groupby(id_column)[score_column].transform('max')
    
    # Filter the rows where the score matches the maximum score for each ID_row
    filtered_df = df[df[score_column] == best_scores].copy(deep=True)
    
    return filtered_df

In [26]:
df_results = filter_best_scores(merged_df, 'index', 'f1')
len(df_results)

333

In [27]:
import numpy as np

for k in results.keys():
    if not k == 'hashcode':
        print(f'{k}',f'\n\tmean: {np.mean(df_results[k])}\n\tstd: {np.std(df_results[k])}\n')

precision 
	mean: 0.8309580647909606
	std: 0.16464016886510133

recall 
	mean: 0.7824891748371067
	std: 0.20287524738079862

f1 
	mean: 0.8046059384718314
	std: 0.1852749454425095



Subtracting input

In [30]:
results_input = bertscore.compute(
    predictions=list(df_results['input']),
    references=list(df_results['version_sin_sesgo']), model_type="bert-base-multilingual-cased")
df_results['precision_input'] = results_input['precision']
df_results['recall_input'] = results_input['recall']
df_results['f1_input'] = results_input['f1']

df_results['precision_diff'] = df_results['precision'] - df_results['precision_input']
df_results['recall_diff'] = df_results['recall'] - df_results['recall_input']
df_results['f1_diff'] = df_results['f1'] - df_results['f1_input']

In [31]:
for k in results.keys():
    if not k == 'hashcode':
        print(f'{k}',f"\n\tmean (diff): {np.mean(df_results[k+'_diff'])}\n\tstd (diff): {np.std(df_results[k+'_diff'])}\n")

precision 
	mean (diff): -0.147619934948357
	std (diff): 0.16816722939475584

recall 
	mean (diff): -0.1812424785024053
	std (diff): 0.2084570808405189

f1 
	mean (diff): -0.16638319252489564
	std (diff): 0.18992517879407717

